In [159]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.metrics import accuracy_score

In [160]:
df = pd.read_csv('../datasets/user_song_mockfeatures.csv')
df['likes_song'] = (df['affinity_score'] > df['affinity_score'].median()).astype(int)

In [204]:


label_encoder = LabelEncoder()
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])


# Perform label encoding on the track_id column
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])
# Move the 'track_id_encoded' column to the first position
df = df[['track_id_encoded'] + [col for col in df.columns if col != 'track_id_encoded']]
df[['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence'] + [col for col in df.columns if col not in ['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence']]]
df.drop(['user_id'], inplace=True, axis=1)
df.head()


,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,intensity_zone,intensity_zone_encoded,cluster,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,6098,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,Zone 4 (Hard),3,10,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,2462,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,Zone 3 (Moderate),2,14,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,1083,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,Zone 5 (Maximum),4,0,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,5252,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,Zone 4 (Hard),3,10,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,5066,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,Zone 5 (Maximum),4,0,0.292775,1,165.0,16.0,83.0,6.27,0


In [162]:
def df_preprocessing(df):
    # Create an instance of LabelEncoder

    model_df = df.copy()
    model_df = model_df.select_dtypes(exclude=['object'])
    model_df = df[['track_id'] + list(model_df.columns)] #creating a dataset with track_id + model_df columns

    return model_df
        

In [163]:
model_df = df_preprocessing(df)
model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id', 'user_id'],inplace=True, axis=1)

/var/folders/sw/6q2vf2dx0r502xm9jxzvdxzc0000gn/T/ipykernel_84065/110666902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id', 'user_id'],inplace=True, axis=1)


In [164]:
model_df.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


In [165]:
# Features and target variable
def dividing_df(df):
    X = model_df.drop(['likes_song', 'affinity_score'], axis=1)
    y = model_df['likes_song']
    categorical_features = ['user_id', 'gender', 'playlist_genre', 'playlist_subgenre', 'key', 'track_artist']
    numerical_features = [col for col in model_df.columns if col not in categorical_features and col not in ['likes_song', 'affinity_score'] and col in model_df.columns]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test
 


In [166]:
X_train, X_test, y_train, y_test = dividing_df(model_df)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [167]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

In [168]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3457
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.498000


In [169]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]


In [170]:
X_test['y_pred'] = y_pred_class
X_test['y_test'] = y_test

In [171]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")

The Mean Squared Error (MSE) on test set: 0.15723165627504812


In [172]:
X_test.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,gender,avg_bpm,age,avg_cadence,avg_pace,y_pred,y_test
50,1843,1698,48,5,2,0.524,0.953,0,-7.097,0.0416,...,4.23,0.166667,0.000,1,211.0,26.0,73.0,5.91,0,1
1077,2239,5621,14,0,18,0.464,0.873,2,-3.924,0.0721,...,3.67,0.000000,0.850,1,126.0,34.0,43.0,6.38,0,0
2325,1677,2782,59,1,12,0.747,0.649,10,-6.218,0.2190,...,3.35,0.000000,0.000,1,167.0,35.0,61.0,5.08,0,0
1474,675,6679,37,3,23,0.814,0.481,5,-6.448,0.1010,...,3.15,0.083333,0.000,0,192.0,19.0,68.0,5.86,1,0
2070,1817,4486,6,0,18,0.648,0.981,0,-2.235,0.1680,...,4.47,0.000000,0.375,1,239.0,27.0,66.0,5.29,0,0


In [173]:
from sklearn.metrics import accuracy_score
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")


The accuracy of the model is: 0.774


In [174]:
feature_importance = gbm.feature_importance()
feature_names = gbm.feature_name()

# Create a dictionary to store the feature importance
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort the feature importance dictionary by value in descending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted feature importance
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")


tempo: 350
loudness: 247
energy: 209
acousticness: 177
danceability: 168
avg_cadence: 167
avg_bpm: 159
avg_pace: 154
speechiness: 152
liveness: 135
duration_minutes: 130
track_id_encoded: 115
age: 115
playlist_subgenre: 106
valence: 103
track_artist: 97
instrumentalness: 92
track_popularity: 90
playlist_genre: 89
key: 56
playlist_name_sentiment: 52
track_name_sentiment: 23
gender: 14


In [175]:
model_df.head()


,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


## Retraining after feature selection

In [180]:
df_2 = model_df.copy()
df_2.drop([col for col, importance in sorted_feature_importance if importance < 90], axis=1, inplace=True)
X_train, X_test, y_train, y_test = dividing_df(df_2)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [181]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000614 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3459
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.495000


In [182]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")

The Mean Squared Error (MSE) on test set: 0.16201073076203815
The accuracy of the model is: 0.77


In [179]:
df_2.head()

,track_id_encoded,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,duration_minutes,affinity_score,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,18,0.675,0.868,-6.360,0.0392,0.00107,0.3740,0.0376,128.007,5.07,0.287000,165.0,16.0,83.0,6.27,0
1,2080,9,0.605,0.584,-7.043,0.1970,0.28600,0.2350,0.4640,125.802,4.09,0.504277,165.0,16.0,83.0,6.27,1
2,521,7,0.673,0.814,-5.411,0.0367,0.01330,0.3430,0.7280,147.193,4.20,0.395855,165.0,16.0,83.0,6.27,0
3,887,11,0.557,0.979,-4.254,0.0489,0.00130,0.0483,0.8560,127.998,4.81,0.285107,165.0,16.0,83.0,6.27,0
4,953,0,0.587,0.977,-3.589,0.0711,0.03530,0.5360,0.5190,146.518,4.46,0.292775,165.0,16.0,83.0,6.27,0


# Generating real data: 
Per user, the input will be: 
- avg_bpm	
- age
- avg_cadence
- avg_pace



In [187]:
#  MOCK VALUES !!!
avg_bpm = 150
age = 30
avg_cadence = 160
avg_pace = 6.5

user_traits = {'avg_bpm':avg_bpm, 'age': age, 'avg_cadence': avg_cadence, 'avg_pace': avg_pace}

In [190]:
df_2 = df_2[['avg_bpm', 'age', 'avg_cadence', 'avg_pace'] + [col for col in df_2.columns if col not in ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']]]
df_2.drop(['likes_song', 'affinity_score'], axis=1, inplace=True)

### Imputting the user's values into the song df

In [191]:
input_columns = ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']
for col in input_columns:
    df_2[col] = user_traits[col]
    
df_2.head()

,avg_bpm,age,avg_cadence,avg_pace,track_id_encoded,track_artist,track_popularity,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_minutes
0,150,30,160,6.5,215,6098,5,18,0.675,0.868,-6.360,0.0392,0.00107,0.715000,0.3740,0.0376,128.007,5.07
1,150,30,160,6.5,2080,2462,62,9,0.605,0.584,-7.043,0.1970,0.28600,0.000000,0.2350,0.4640,125.802,4.09
2,150,30,160,6.5,521,1083,68,7,0.673,0.814,-5.411,0.0367,0.01330,0.002930,0.3430,0.7280,147.193,4.20
3,150,30,160,6.5,887,5252,0,11,0.557,0.979,-4.254,0.0489,0.00130,0.834000,0.0483,0.8560,127.998,4.81
4,150,30,160,6.5,953,5066,33,0,0.587,0.977,-3.589,0.0711,0.03530,0.000005,0.5360,0.5190,146.518,4.46


### Predicting for that specific user

In [197]:
y_pred = gbm.predict(df_2, num_iteration=gbm.best_iteration, predict_disable_shape_check=True)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

### Retrieving the song name and the pred value

In [207]:
merged_df = df.merge(df_2[['track_id_encoded', 'y_pred']], how='left', on='track_id_encoded', suffixes=('', '_y'))
merged_df.drop(['likes_song', 'affinity_score', 'gender'], axis=1, inplace=True)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
merged_df[['track_name', 'y_pred' ]]


,track_name,y_pred
0,Animals - Original Mix,0
1,Cater 2 U,0
2,White Wedding - Pt. 1,0
3,Outlaw - Original Mix,0
4,Misty Night Cruising,0
...,...,...
2769,Keep On,0
2770,Keep On,0
2771,Sad (feat. Afrojack),0
2772,Call The Parade,0


### To get real data: each user has to corroborar que el prediction era real, y si no hay que cambiarlo. 